##### Load Dependencies

In [ ]:
import os
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler
from pathlib import Path
from playwright.sync_api import sync_playwright
from agents import Agent, Runner
import tiktoken
import base64
import requests
from sklearn.model_selection import train_test_split
from static import GptRequests, GptRoles, GptLanguages
from booger import Error, ErrorDialog, ChatWindow, FileDialog, FileBrowser
from importlib import reload

# Pinecone

##### Initialize Client

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone, ServerlessSpec

# Initialize a Pinecone client with your API key
pinecone_key = os.getenv( 'PINECONE_API_KEY' )
pc = Pinecone( api_key=pinecone_key )

### Upsert Data

##### Create Serverless Index

In [ ]:
index_name = "quickstart"

pc.create_index( name=index_name,
    dimension=1024, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

##### Create Vector Embeddings

In [ ]:
##### Create Vector Embeddings
data = [
	{ "id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture." },
	{ "id": "vec2",
	  "text": "The tech company Apple is known for its innovative products like the iPhone." },
	{ "id": "vec3", "text": "Many people enjoy eating apples as a healthy snack." },
	{ "id": "vec4",
	  "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces." },
	{ "id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes." },
	{ "id": "vec6",
	  "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership." }
]

embeddings = pc.inference.embed(
	model="multilingual-e5-large",
	inputs=[ d[ 'text' ] for d in data ],
	parameters={ "input_type": "passage", "truncate": "END" }
)
print( embeddings[ 0 ] )

##### Upsert Data

In [ ]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

##### Check Index

In [ ]:
print(index.describe_index_stats())